In [11]:
import sys
import os
import kagglehub
import matplotlib.pyplot as plt
import numpy as np

In [2]:
!git clone https://github.com/maximus3343/movie-rec

Cloning into 'movie-rec'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 38.96 KiB | 3.25 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
cloned_repo_path = '/content/movie-rec'

if cloned_repo_path not in sys.path:
    sys.path.append(cloned_repo_path)
print(f"Added {cloned_repo_path} to sys.path")

Added /content/movie-rec to sys.path


In [4]:
import pandas as pd

from MovieEnv import MovieEnv
from PPO import PPO
from Buffer import Experience

In [5]:
path = kagglehub.dataset_download("vodclickstream/netflix-audience-behaviour-uk-movies")
dataset_name = os.listdir(path)[0]
path = os.path.join(path, dataset_name)

100%|██████████| 18.1M/18.1M [00:00<00:00, 140MB/s]

Extracting files...


In [ ]:
def main():
    df = pd.read_csv(path)
    N = 4  # Number of last movies watched kept as state.
    envi = MovieEnv(df, N=N, alpha=1, beta=0)

    lr = 1e-4  # Same learning rate for all modules.
    gamma = 0.95  # Discount factor.
    in_features = 128
    hidden_size = 256
    embedding_dim = 64
    id_list = df['movie_id'].unique()  # List of movies.
    K_epochs = 5
    eps_clip = 0.2
    top_k = 0

    a = PPO(in_features=in_features, out_features=envi.action_space.n, hidden_size=hidden_size, embedding_dim=embedding_dim, lr_actor=lr,
            lr_critic=lr, lr_sr=lr, gamma=gamma, N=N, K_epochs=K_epochs, eps_clip=eps_clip, movie_id_list=id_list, top_k=top_k,
            algo='monte_carlo')

    episodes = 500
    rewards = []

    for k in range(episodes):
        state, info = envi.reset()
        done = False

        total_reward = 0

        while not done:
            action, state_value, action_logprob = a.select_action(state)
            next_state, reward, done, _, _ = envi.step(action)

            # exp = Experience(state, action, reward, next_state, done, state_value, action_logprob)
            a.add_experience(Experience(state, action, reward,
                            next_state, done, state_value, action_logprob))

            total_reward += reward

            state = next_state

        a.learn()
        rewards.append(reward)

    return rewards

if __name__ == "__main__":
    rewards = main()
    plt.plot(rewards,np.arange(len(rewards)))
    plt.title("Rewards per Episode")
    plt.xlabel("Episode")
    plt.ylabel("Reward")



The project aims to train an intelligent agent to recommend movies to a simulated user. It frames the movie recommendation task as a MDP using a custom environment based on a static dataset, Netflix Behaviour in the UK.